# **tcheasy** with dict declarations

**tcheasy** is able to receive and work with a completely custom dict.<br> The only thing you have to do is to write all those declarations of your parameters. <br>
This looks like a hughe effort, but believe me, once in flesh and blood and you can do powerfull stuff! <br>
<br>
But before we dive in some basics (and the import!):

In [1]:
# import tcheasy
from tcheasy import tcheasy

<br>
<br>

## tcheasy's dict structure

The structure of the dict is always the same. There are basically three main keywords:
- `positional`: All parameters that are not declared as \*args or \*\*kwargs.
- `args`: All parameters that are not explicit defined within your function (--> the typical `, *args` solution during defining your function).
- `kwargs`: All keyword bounded parameters which are not explicit defined whint your function (--> the `, **kwargs` part of your function).

Each of those keywords accept a specific dict as value. <br>
The cool about it: The structure of the dict for `positional` and `kwargs` is the same! <br>
<br>

<br>

## `positional` & `kwargs` dict structure

Let's look first at the structure of the of `positional` and `kwargs`:

In [2]:
# an example dict for an parameter called 'your_parameter'
pos_kwargs = {
    'your_parameter':{
        'type':int,
        'default':5,
        'restriction':"value > 4"
    }
}

As you can see, the dict is a nested dict. In within this dict, you can specify for each of your parameters the structure. <br>
The parameter name is the key (in the example the `your_parameter`) and the value is the declaration. <br>
I will tell what the declaration actually means in a second, but first lets have a look at the inner keywords. These are: 
- `type`: the python type of the parameter
- `default`: the default value if no parameter is passed (this will actually overwrite the functions default!)
- `restriction`: The boundries the parameter input has to meet.

<br>

The only mandatory keyword is the `type`. The rest is optional:

In [3]:
# create a declaration without 'type':
toCheck = {
    'positional':{
        'a':{
            'default':5
        }
    }
}

# create the function
@tcheasy(toCheck)
def will_not_work(a):
    """Will not work! """
    return "You will never see this!"

AssertionError: Missing 'type' definition in 'positional': 'a'.

<p><small><em> Sorry for all of you who will try to run the notebook in one go! </em></small></p> <br>
<br>
As you can see, **tcheasy** throws an exception, telling you that it misses the `type` definition for the `positional` parameter `a`. <br>
<br>
Now that we saw how to not do it, lets do it the right way:

In [4]:
# create a correct declaration
toCheck = {
    'positional':{
        'a':{
            'type':int,
            'restriction':"value > 5"
        }
    }
}

# create the function
@tcheasy(toCheck)
def will_work(a):
    """This will work! """
    return a

See? No exception (although we did remove the keyword `default`).

In [5]:
# call the function
will_work(4)

{'success': False,
 'error': "[K.3]: The parameter 'a' does not meet the restriction 'value > 5'. Value is currently '4'."}

Uff.. But why did we not get the parameter input? <br>
Well, you did pass a int value which did not meat the restriction. (Remember, we did specify the `restriction` to be `value > 5`... and 4 is definitely not greater!). <br>
<br>
The `restrictions` is actually built in the way, that you can pass an condition check like you would in an `if` statement. The only exception is, that **tcheasy** inserts the passed function parameter at runtime into the `value` keyword. <br>
<br>
So, in our example, the statement `value > 5` becomes during runtime `assert(a > 5)`. <br>
<br>
Let's try it:

In [6]:
will_work(a=6)

6

As I told ya! As soon as `a` meets the restriction, the function runs. <br>
Once you master this concept you can build pretty complex restrictions! <br>
<br>
Rember! `positional` and `kwargs` are built with the same structure:

In [7]:
# a full example with multiple types
toCheck = {
    'positional':{
        'first_param':{
            'type':bool
        },
        'second_param':{
            'type':int,
            'restriction':"value == 3",
            'default':3
        }
    },
    'kwargs':{
        'z':{
            'type':float,
            'restriction':"value > .5"
        }
    }
}

Now switch to `args`! <br>
<br>
<br>

## The structure of `args`

First things first: <br>
`args` uses the same keywords to declare its parameters! <br>
But why bother with a new structure then? <br>
Well, \*args are by nature without any keywords (e.g. `some_function(pos, pos, args, args, args, kwargs=value)` ). Therefore they are order sensitive. And this is represented by the structure (aka form follows function!). <br>
<br>
Within the `args` each parameter is put into a list:

In [8]:
# only 'args' with multiple keywords:

only_args:{
    'args':[
        {'type':int, 'default':5, 'restriction':"value < 2"},
        {'type':bool},
        {'type':float, 'restriction': "value < -2."}
    ]
}

As you can see, the declaration is basically the same dict as for `positional` and `kwargs` but lives now inside a list. And this list has to be sorted according to the appearance your potential \*args!

In [9]:
# a example dict with two parameters
toCheck = {
    'args':[{'type':int}, {'type':bool}]
}

# create the function
@tcheasy(toCheck)
def only_args(*args):
    """Function returns the args """
    return args

In [10]:
# call it
only_args(5, 5)

{'success': False,
 'error': "[A.2]: The '*args' parameter at position '1' needs to be a(n) bool."}

Each single \*args was checked. Note that the error msg tells you that the parameter at position 1 is wrong. <br>
**tcheasy** is not able to name the parameter because \*args are never keyworded. Therefore it counts - beginning from the first \*args onward - from zero to your positional idx. <br>
To show you this in another example:

In [11]:
# new function, but this time with positionals
@tcheasy(toCheck)
def mixed(a, b, *args):
    """Function with additional positionals """
    return True

In [12]:
# call it with wrong args (args will be: 10, 15); 15 is no bool!
mixed(1,"string", 10, 15)

{'success': False,
 'error': "[A.2]: The '*args' parameter at position '1' needs to be a(n) bool."}

Even though the incorrect \*args is on idx 3, it is relativ to the \*args (in our example the (10,15)) the first idx. <br>
<br>
The previous example shows you also another core feature of **tcheasy**: the use of mixed dicts!

<br>
<br>

# Mixed input checks
You do not have to define everytime every 'block'. Sometimes you only want to check certain inputs. <br>
In these cases, you just write the parameter type 'block' for the specific parameter group. <br>
<br>
For example: If you want only to check the \*args (like in the previous example) you only pass a dict with `args` defined. <br>
<br>
There is one minor exception to keep in mind though: <br>
Once a block is defined, you have to define every declared function variable! <br>
So this does not work:

In [13]:
# create a dict with only positionals
toCheck = {
    'positional':{
        'a':{'type':int}
    }
}

# create a function with positionals a & b and some **kwargs
@tcheasy(toCheck)
def mixed_not_working(a, b, **kwargs):
    """Does not work! """
    return "You will never see this!"

In [14]:
# call it
mixed_not_working(a=15, b="some-string", kwargs_arg="some-kwargs")

{'success': False,
 'error': "[K.0]: Your passed parameter 'b' was not expected."}

In this case **tcheasy** did check if you expected a parameter `b` as a `positional` (but you did not, as your `positional` declaration stated). <br>
This feature is kinda useless for `positional`s but interesting for `args` & `kwargs`. <br>
<br>
Consider for example the following example: <br>
You just want to pass the input of one function as \*\*kwargs to another. After reworking some parts of your follow up function, you see that the needed parameter changed. In this case you just pass another dict to **tcheasy** and you are done:

In [15]:
# these are the parameters which are passed to your follow up
params = {'omega':5, 'theta':"someString"}

# this is the original declaration dict
toCheck = {
    'kwargs':{
        'omega':{'type':int},
        'theta':{'type':str}
    }
}

# create your original follow up function
@tcheasy(toCheck)
def follow_up(**kwargs):
    """Return the passed kwargs"""
    return kwargs

In [16]:
# call it as intended
follow_up(**params)

{'omega': 5, 'theta': 'someString'}

In [17]:
# Now you change your 'follow_up' and need another variable
params = {'omega':5, 'theta':"someString", 'alpha':.1}

# call the function
follow_up(**params)

{'success': False,
 'error': "[K.0]: Your passed parameter 'alpha' was not expected."}

In [18]:
# changed inputs? Lets add the new one with an restriction
toCheck = {
    'kwargs':{
        'omega':{'type':int},
        'theta':{'type':str},
        'alpha':{'type':float, 'restriction':"value ==.1"}
    }
}

# just the copy of the above function
@tcheasy(toCheck)
def follow_up(**kwargs):
    """Return the passed kwargs"""
    return kwargs

In [19]:
# call the function again
follow_up(**params)

{'omega': 5, 'theta': 'someString', 'alpha': 0.1}

You can also use a mix of python type hinting (for `positionals`) and the dict:

In [20]:
# a dict with only args
toCheck = {
    'args':[{'type':float}]
}

# create a function with positionals and *args
@tcheasy(toCheck)
def pos_and_args(a, b:str = "default", *args):
    """A mixed type function """
    return {'a':a, 'b':b, 'args':args}

In [21]:
# call the function
pos_and_args(15, "notTheDefault", .5)

{'a': 15, 'b': 'notTheDefault', 'args': (0.5,)}

In [22]:
# call the function with wrong type for b
pos_and_args(15, 1, .5)

{'success': False, 'error': "[K.2]: The parameter 'b' needs to be a(n) str."}

In [23]:
# call the function with wrong type for the args param
pos_and_args(15, "string", 5)

{'success': False,
 'error': "[A.2]: The '*args' parameter at position '0' needs to be a(n) float."}

<br>
<br>

# Additional info

There are some additional info to keep in mind:

## Function defaults overwrit tcheasy defaults

In [24]:
# function declared defaults overwrite tcheasy defaults
toCheck = {
    'positional':{
        'a':{'type':int, 'default':10}
    }
}

# create a function with positionals and *args
@tcheasy(toCheck)
def overwritten(a=15):
    """Return passed parameter """
    return a

In [25]:
# call it without passing a
overwritten()

15

## Hints only used if `positional` is not provided

In [26]:
# hints only apply if 'positionals' not provided
# function declared defaults overwrite tcheasy defaults
toCheck = {
    'positional':{
        'a':{'type':int}
    }
}

# create a function with positionals and *args
@tcheasy(toCheck)
def do_not_use_hints(a:str):
    """Return passed parameter """
    return a

In [27]:
# call it --> a is not a string!
do_not_use_hints(15)

15

## Multiple possible types

In [28]:
# define multiple types in the tcheasy dict
toCheck = {
    'positional':{
        'a':{'type':(int, float)}
    }
}

# create a function with positionals and *args
@tcheasy(toCheck)
def multiple_types(a):
    """Return passed parameter """
    return a

In [29]:
# call a as str
multiple_types("a as string")

{'success': False,
 'error': "[K.2]: The parameter 'a' needs to be a(n) int | float."}

In [30]:
# call a as float
multiple_types(.10510)

0.1051

In [31]:
# call a as int
multiple_types(696)

696

## Use None as type(None)

In [32]:
# define input of variable as None
toCheck = {
    'args':[{'type':None}]
}

# create a function with positionals and *args
@tcheasy(toCheck)
def with_None(*args):
    """Return passed parameter """
    return True

AssertionError: If you want to specify the 'type' None you have to pass it as type(None). Error in 'args' at position: '0'.

In [33]:
# this time its correct
toCheck = {
    'args':[{'type':type(None)}]
}

# create a function with positionals and *args
@tcheasy(toCheck)
def with_None(*args):
    """Return passed parameter """
    return True

<br>
<br>

# That's it!
Now you are ready to use **tcheasy**'s dict-type checks! <br>
<br>
If you need at any point help, just call `help(tcheasy)`